In [1]:
# Sample program for one scrip
from ib_insync import *
util.startLoop()
ib=IB().connect('127.0.0.1', 7496, clientId=9) # kavi tws live
# ib = IB().connect('127.0.0.1', 4001, clientId=9) # kavi IBG live

In [ ]:
%%time
import numpy as np
import pandas as pd
from itertools import product

# sd multiple for band
sigma = 2   # 2 sigma is about 95% probability

# market
exchange = 'NSE'

#... prepare lot dataframe for underlying
# from 5paisa
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
df_paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol')

# Rename Symbol and Margin fields
df_paisa = df_paisa.rename(columns={'Symbol': 'nseSymbol', 'TotMgn%': 'marginpct'})

# Convert columns to numeric and make margin to pct
df_paisa = df_paisa.apply(pd.to_numeric, errors='ignore')
df_paisa.marginpct = df_paisa.marginpct.div(100)

# Truncate to 9 characters for ibSymbol
df_paisa['ibSymbol'] = df_paisa.nseSymbol.str.slice(0,9)

# nseSymbol to ibSymbol dictionary for conversion
ntoi = {'M&M': 'MM', 'M&MFIN': 'MM', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# remap ibSymbol, based on the dictionary
df_paisa.ibSymbol = df_paisa.ibSymbol.replace(ntoi)

# %%time
#... Get the scrip
symbol = 'ACC'
contract = Stock('ACC', 'NSE')
ib.qualifyContracts(contract)

#... Get the price from bar data 
# Last second's bar of historical with realtime updates
undPrice = ib.reqHistoricalData(contract=contract, endDateTime='', 
                     durationStr='60 S', barSizeSetting='1 secs', 
                     whatToShow='TRADES', useRTH=True, 
                     formatDate=1, keepUpToDate=True)[-1].high

#... Get the volatility
duration = '12 M'
size = '1 day'
bars = ib.reqHistoricalData(contract=contract, endDateTime='', 
                     durationStr=duration, barSizeSetting=size, 
                     whatToShow='MIDPOINT', useRTH=True, 
                     formatDate=1, keepUpToDate=True)
stDev = np.std(a=[b.high for b in bars], ddof=0)

#... Get the lot and margin
lot = df_paisa.loc[df_paisa.ibSymbol == symbol, 'Mlot'].item()
margin = df_paisa.loc[df_paisa.ibSymbol == symbol, 'TotMgnPerLt'].item()

#... Get the option chain
xk = ib.reqSecDefOptParams(underlyingSymbol=contract.symbol, 
                      futFopExchange='', 
                      underlyingConId=contract.conId, underlyingSecType=contract.secType)

#... Weed out unwanted SDs
sigma = 2   # 2 sigma is about 95% probability

safek = sorted([s for k in xk 
    for s in k.strikes 
    if (s < (undPrice - stDev*sigma)) | (s > (undPrice + stDev*sigma))])

tups = [(sorted(i.expirations), safek) for i in xk]

tups_xk = [p for tup in tups for p in product(*tup)]

#... Get the margins and lots

# make dataframe for the option contracts
df = pd.DataFrame(tups_xk, columns = ['expiry', 'strike'])

df['ibSymbol'] = symbol
df['exchange'] = exchange
df['lot'] = lot # miniumum margin
df['margin'] = margin # margin per lot

# puts for strike below undPrice and calls for strike above undPrice
df['right'] = np.where(df.strike < undPrice, 'P', 'C')

# list of options
opt_list = [Contract(symbol=symbol, secType='OPT', exchange=exchange, strike=strike,
       lastTradeDateOrContractMonth=expiry, right=right)
for symbol, strike, expiry, right in zip(df.ibSymbol, df.strike, df.expiry, df.right)]

# qualified contracts
contracts = [c for i in range(0, len(opt_list), 50)
             for c in ib.qualifyContracts(*opt_list[i: i+50])]

tickers = ib.reqTickers(*contracts)
# ib.sleep(2)

In [21]:
# bid, ask and tickers for contracts
df1 = pd.DataFrame({ticker.contract.conId: 
          (ticker.contract.lastTradeDateOrContractMonth, ticker.contract.strike, 
           ticker.contract.symbol, ticker.contract.right,
           ticker.bid, ticker.ask, ticker.close, ticker.modelGreeks.delta, ticker.modelGreeks.optPrice, ticker) 
          for ticker in tickers}).T.reset_index()


df1.columns=['conId', 'expiry', 'strike', 'ibSymbol', 'right', 
             'bid', 'ask', 'close', 'delta', 'modelPrice', 'ticker']

# replace -1s in bid and ask with np.nan
df1[['bid', 'ask']] = df1[['bid', 'ask']].replace({-1: np.nan})

cols_to_use = df.columns.difference(df1.columns)
df2 = pd.concat([df1, df[cols_to_use]], axis=1, join='inner')  # removes scrips without tickers

In [24]:
df2[df2.close != 0]

,conId,expiry,strike,ibSymbol,right,bid,ask,close,delta,modelPrice,ticker,exchange,lot,margin
14,335374757,20181227,1220,ACC,P,NaN,NaN,0.05,-0.000153208,0.00333214,"Ticker(contract=Contract(secType='OPT', conId=...",NSE,400,105464.0
15,335374987,20181227,1800,ACC,C,NaN,NaN,0.05,0.000406798,0.00835548,"Ticker(contract=Contract(secType='OPT', conId=...",NSE,400,105464.0
16,335374992,20181227,1820,ACC,C,NaN,NaN,0.05,0.000188737,0.00369739,"Ticker(contract=Contract(secType='OPT', conId=...",NSE,400,105464.0
17,335375002,20181227,1840,ACC,C,NaN,NaN,0.05,8.48528e-05,0.00158881,"Ticker(contract=Contract(secType='OPT', conId=...",NSE,400,105464.0
18,335375011,20181227,1860,ACC,C,NaN,NaN,0.05,3.70005e-05,0.000663492,"Ticker(contract=Contract(secType='OPT', conId=...",NSE,400,105464.0
35,339361752,20190131,1140,ACC,P,NaN,NaN,0.6,-0.00929933,0.603278,"Ticker(contract=Contract(secType='OPT', conId=...",NSE,400,105464.0
36,339361761,20190131,1160,ACC,P,NaN,NaN,0.9,-0.012641,0.838864,"Ticker(contract=Contract(secType='OPT', conId=...",NSE,400,105464.0
37,339361771,20190131,1180,ACC,P,NaN,NaN,1.3,-0.0173345,1.18568,"Ticker(contract=Contract(secType='OPT', conId=...",NSE,400,105464.0
38,339361776,20190131,1200,ACC,P,NaN,NaN,1.9,-0.0237475,1.6844,"Ticker(contract=Contract(secType='OPT', conId=...",NSE,400,105464.0
39,339361786,20190131,1220,ACC,P,NaN,NaN,2.7,-0.0322663,2.3844,"Ticker(contract=Contract(secType='OPT', conId=...",NSE,400,105464.0


In [25]:
undPrice

1507.15

In [26]:
ib.reqMktData(contract=contracts[57], genericTickList=456)

Ticker(contract=Contract(secType='OPT', conId=343974499, symbol='ACC', lastTradeDateOrContractMonth='20190228', strike=1220.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC19FEB1220PE', tradingClass='ACC'), time=datetime.datetime(2018, 12, 19, 0, 11, 53, 183301, tzinfo=datetime.timezone.utc), bid=-1.0, bidSize=0, ask=-1.0, askSize=0, close=7.7, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[], modelGreeks=OptionComputation(impliedVol=0.3557415689877849, delta=-0.06931697997440212, optPrice=7.702266703988192, pvDividend=0.0, gamma=0.0005641256999896126, vega=0.8848648121607829, theta=-0.19976201342714275, undPrice=1501.25))